# Lecture 1 Hands-On: Poisson Warm-Up

This notebook pairs with Lecture 1. Start with a plain NumPy simulation of Poisson counts, then step into a SimPy arrival process.

## Setup

In [ ]:
import pathlib

import numpy as np

try:
    import simpy
except ImportError as exc:
    raise SystemExit("SimPy is required for this notebook. Install via 'pip install simpy'.") from exc

RNG = np.random.default_rng(42)
NOTEBOOK_DIR = pathlib.Path.cwd()


---
## Part 1 — Plain Python Warm-Up
Follow the checklist from the slides:
1. Set the hourly rate `lam` and scale it to a half-hour window.
2. Simulate `n_windows = 1_000` Poisson counts for the 30 minute window.
3. Report the sample mean and variance versus the theoretical value `lambda_window`.
4. Estimate `P(X >= 1)` empirically by counting non-zero draws.
5. Summarise the takeaways in the Markdown cell below.

In [ ]:
lam_per_hour = 4
lambda_window = lam_per_hour * 0.5  # TODO: confirm the scaling
n_windows = 1_000

# TODO: simulate Poisson counts and compute sample mean, variance, and probability of at least one event
counts = None
sample_mean = None
sample_var = None
p_ge_one = None

sample_mean, sample_var, p_ge_one


### Notes on Part 1
*Replace this bullet list with a brief comparison between simulation and theory.*

---
## Part 2 — SimPy Arrival Stream
Build the arrival process generator and collect arrival times for two hours (8 half-hour windows). Steps:
1. Define `arrival_process(env, lam, log)` that loops forever, sampling exponential inter-arrival times using `np.random.exponential(1 / lam)` and appending `env.now` to `log`.
2. Create a simple driver function that runs the environment until `duration=2` hours.
3. After the run, compute empirical inter-arrival times and compare them to the exponential mean (`1 / lam`).
4. Plot a histogram of inter-arrival times and overlay the theoretical exponential density.

In [ ]:
lam = 4  # events per hour
duration_hours = 2


In [ ]:
# TODO: implement the arrival process and simulation driver
def arrival_process(env, lam, log):
    raise NotImplementedError

def simulate_arrivals(lam, duration_hours):
    raise NotImplementedError

arrival_log = simulate_arrivals(lam, duration_hours)
arrival_log[:5]


### Analyse the Arrival Log
*Compute summary statistics and visual diagnostics here (e.g., mean inter-arrival vs. `1 / lam`).*

In [ ]:
# TODO: derive inter-arrival times, compute summary statistics, and produce at least one plot


### Stretch Goal
Upgrade the arrival process into an M/M/1 queue by adding a service process and tracking waiting times. Document any assumptions you make.